In [ ]:
import polars as pl

df = pl.read_ndjson("data/gym/dump_prompt_solution_gym_0605_e.jsonl")

df[0]

In [ ]:
df2 = df.filter(pl.col("title").str.contains(r"[^\x00-\x7F]") & 
                ~pl.col("title").str.contains(r"^[A-Za-z0-9\s\p{P}]*$"))

len(df2)

In [ ]:
list(df2["title"])

In [17]:
sub_df2 = df2.filter(pl.col("prompt").str.contains(r"[^\x00-\x7F]") & 
                ~pl.col("prompt").str.contains(r"^[A-Za-z0-9\s\p{P}]*$"))

diff_df2 = df2.filter(~(pl.col("prompt").str.contains(r"[^\x00-\x7F]") & 
                ~pl.col("prompt").str.contains(r"^[A-Za-z0-9\s\p{P}]*$")))

In [ ]:
list(diff_df2["prompt"])

In [ ]:
df3 = df.filter(pl.col("prompt").str.contains(r"[^\x00-\x7F]") & 
                ~pl.col("prompt").str.contains(r"^[A-Za-z0-9\s\p{P}]*$"))

len(df3)

In [ ]:
from langdetect import detect
dfn = df.with_columns(pl.col("prompt").map_elements(lambda s: detect(s) if s else "EMPTY").alias("lang"))

dfn["lang", "prompt"]

In [29]:
df_not_en = dfn.filter(pl.col("lang") != "en")

# START FROM HERE:

In [1]:
# Import required libraries and load data
import polars as pl
import re
from langdetect import detect

# Load the dataframes
dfb = pl.read_ndjson("data/gym/dump_prompt_solution_gym_0605_b.jsonl")
dfc = pl.read_ndjson("data/gym/dump_prompt_solution_gym_0605_c.jsonl")
dfd = pl.read_ndjson("data/gym/dump_prompt_solution_gym_0605_d.jsonl")
dfe = pl.read_ndjson("data/gym/dump_prompt_solution_gym_0605_e.jsonl")

In [19]:
def remove_tags(s: str):
    return re.sub(r'<[/]?\w+>', '', s)

def detect_lang(s: str):
    a = remove_tags(s)
    try:
        if a == "":
            return "EMPTY"
        return detect(a)
    except Exception as e:
        print(a, e)
    return "idk"

# Function to add language detection and split dataframe
def split_by_lang(df):
    # Add language detection
    df_with_lang = df.with_columns(
        pl.col("prompt").map_elements(
            lambda s: detect_lang(s) if s else "EMPTY", 
            return_dtype=pl.Utf8
        ).alias("lang"),
        pl.col("title").map_elements(
            lambda s: detect_lang(s) if s else "EMPTY",
            return_dtype=pl.Utf8
        ).alias("lang_title"),
    )
    
    # Split into English/Empty and non-English/non-Empty
    df_en = df_with_lang.filter((pl.col("lang") == "en") | (pl.col("lang") == "EMPTY") | (pl.col("lang_title") == "en"))
    df_non_en = df_with_lang.filter((pl.col("lang") != "en") & (pl.col("lang") != "EMPTY") & (pl.col("lang_title") != "en"))
    
    return df_en, df_non_en

In [20]:
# Process each dataframe
dfb_en, dfb_non_en = split_by_lang(dfb)
dfc_en, dfc_non_en = split_by_lang(dfc)
dfd_en, dfd_non_en = split_by_lang(dfd)
dfe_en, dfe_non_en = split_by_lang(dfe)

# Print statistics for each split
for name, df_en, df_non_en in [
    ("dfb", dfb_en, dfb_non_en),
    ("dfc", dfc_en, dfc_non_en),
    ("dfd", dfd_en, dfd_non_en),
    ("dfe", dfe_en, dfe_non_en)
]:
    total = len(df_en) + len(df_non_en)
    print(f"\n{name} statistics:")
    print(f"Total rows: {total}")
    print(f"English/Empty: {len(df_en)} ({len(df_en)/total*100:.1f}%)")
    print(f"Non-English: {len(df_non_en)} ({len(df_non_en)/total*100:.1f}%)")
    
    if len(df_non_en) > 0:
        print("Sample of non-English languages:", df_non_en.select("lang").unique().to_series().to_list())


810975 No features in text.
+ - No features in text.
42 No features in text.
2300 No features in text.
2024 No features in text.
0:0 No features in text.
= 10 No features in text.
7485 No features in text.
0689 No features in text.
01 No features in text.
2016 No features in text.

dfb statistics:
Total rows: 3000
English/Empty: 2687 (89.6%)
Non-English: 313 (10.4%)
Sample of non-English languages: ['id', 'da', 'fr', 'it', 'tl', 'vi', 'ko', 'ru', 'ca', 'et', 'no', 'ro', 'af', 'es', 'tr', 'zh-cn', 'pt', 'sk']

dfc statistics:
Total rows: 3000
English/Empty: 2670 (89.0%)
Non-English: 330 (11.0%)
Sample of non-English languages: ['no', 'cy', 'bn', 'uk', 'zh-cn', 'tr', 'vi', 'ca', 'fr', 'et', 'id', 'ru', 'da', 'es', 'pt', 'de', 'bg', 'af']

dfd statistics:
Total rows: 3000
English/Empty: 2743 (91.4%)
Non-English: 257 (8.6%)
Sample of non-English languages: ['ru', 'ca', 'da', 'sk', 'bn', 'pt', 'es', 'cy', 'vi', 'no', 'hr', 'et', 'id', 'zh-cn', 'fr']

dfe statistics:
Total rows: 580
English/

In [16]:
# I want to TRANSFER rows in non_en data frames that contain `iframe src=` in the `prompt` column to the en data frames

dfb_non_en_ifr = dfb_non_en.filter(pl.col("prompt").str.contains("iframe src="))
dfc_non_en_ifr = dfc_non_en.filter(pl.col("prompt").str.contains("iframe src="))
dfd_non_en_ifr = dfd_non_en.filter(pl.col("prompt").str.contains("iframe src="))
dfe_non_en_ifr = dfe_non_en.filter(pl.col("prompt").str.contains("iframe src="))

# Combine these non_en_ifr rows back into the en data frames

dfb_en = dfb_en.vstack(dfb_non_en_ifr)
dfc_en = dfc_en.vstack(dfc_non_en_ifr)
dfd_en = dfd_en.vstack(dfd_non_en_ifr)
dfe_en = dfe_en.vstack(dfe_non_en_ifr)

# And remove these rows from the non_en data frames

dfb_non_en = dfb_non_en.filter(~pl.col("prompt").str.contains("iframe src="))
dfc_non_en = dfc_non_en.filter(~pl.col("prompt").str.contains("iframe src="))
dfd_non_en = dfd_non_en.filter(~pl.col("prompt").str.contains("iframe src="))
dfe_non_en = dfe_non_en.filter(~pl.col("prompt").str.contains("iframe src="))

In [18]:
dfb_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_b_en.jsonl")
dfc_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_c_en.jsonl")
dfd_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_d_en.jsonl")
dfe_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_e_en.jsonl")

dfb_non_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_b_non_en.jsonl")
dfc_non_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_c_non_en.jsonl")
dfd_non_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_d_non_en.jsonl")
dfe_non_en.write_ndjson("data/op/dump_prompt_solution_gym_0605_e_non_en.jsonl")

In [ ]:
dftest = dfb_en.filter(pl.col("prompt_id").str.contains("Gym/370346F"))
detect(str(dftest["prompt"]))